In [ ]:
import paper_dataloaders
import paper_models

from seqdata.core import *
from seqdata.models.core import *
from seqdata.learner import *
from seqdata.dataloaders import *
from seqdata.dualrnn import *
from seqdata.hpopt import *
from fastai.basics import *
from fastai.callback.all import *
from seqdata.quaternions import *
from pathlib import Path
from ray import tune
import pickle

from paper_dataloaders import *
from paper_models import *

In [ ]:
for id in dict_dls_fransys.keys():
    print(id)
    dls = dict_dls_fransys[id](bs=1024)  

In [ ]:
for id in dict_dls_fransys_test.keys():
    print(id)
    dls = dict_dls_fransys[id](bs=1024)  


In [ ]:
with open( 'dls_normalize.p', "rb" ) as f:
            dls_normalize_values = pickle.load( f )

In [ ]:
dls_normalize_values

In [ ]:
dl = 'Ship'
dls = dict_dls_fransys[dl]()
n_u, n_y, n_x, init_sz, _ = get_dls_parameters(dict_dls_fransys[dl])
model = model_narprog_small(n_u,n_x,n_y,init_sz)
lrn = Learner(dls,model,loss_func=SkipNLoss(nrmse_std,init_sz),
                metrics=[SkipNLoss(rmse,init_sz),
                        SkipNLoss(nrmse_std,init_sz),
                        SkipNLoss(nrmse,init_sz)])
lrn.fit_flat_cos(30,lr=3e-3,pct_start=0.2)

In [ ]:
dl = 'Robot'
dls = dict_dls_fransys[dl](pred_sz=60)
n_u, n_y, n_x, init_sz, _ = get_dls_parameters(dict_dls_fransys[dl])
model = model_narprog_small(n_u,n_x,n_y,init_sz)
lrn = Learner(dls,model,loss_func=SkipNLoss(mae,init_sz),
                metrics=[SkipNLoss(rmse,init_sz),
                        SkipNLoss(nrmse_std,init_sz),
                        SkipNLoss(nrmse,init_sz)])
lrn.add_cb(TbpttResetCB())
lrn.fit_flat_cos(30,lr=3e-3,pct_start=0.2)

In [ ]:
dls = dict_dls_fransys[dl]()
lrn = Learner(dls,model,loss_func=SkipNLoss(mae,init_sz),
                metrics=[SkipNLoss(rmse,init_sz),
                        SkipNLoss(nrmse_std,init_sz),
                        SkipNLoss(nrmse,init_sz)])
lrn.show_results(max_n=3)

In [ ]:
dls = dict_dls_fransys_test[dl](valid_stp_sz=1)
# dls = create_dls_ship_prediction_test_ood()
lrn = Learner(dls,model,loss_func=SkipNLoss(mae,init_sz),
                metrics=[SkipNLoss(rmse,init_sz),
                        SkipNLoss(nrmse_std,init_sz),
                        SkipNLoss(nrmse,init_sz)])
lrn.validate()

In [ ]:
dls = dict_dls_fransys_test[dl](pred_sz=60,valid_stp_sz=60)
# dls = create_dls_ship_prediction_test_ood()
lrn = Learner(dls,model,loss_func=SkipNLoss(mae,init_sz),
                metrics=[SkipNLoss(rmse,init_sz),
                        SkipNLoss(nrmse_std,init_sz),
                        SkipNLoss(nrmse,init_sz)])
lrn.add_cb(TbpttResetCB())
lrn.validate()

In [ ]:
lrn.show_results(max_n=3)

In [ ]:
lrn.validate()